In [ ]:
import numpy as np

iterations_number = 100
X0 = np.array([0.73, -0.23])
optimal_x = np.array([-np.log(2) / 2.0, 0.0])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rcParams["figure.figsize"] = [12,12]
def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
def animate_trajectory(traj):
    fig, ax = plt.subplots()
    n = len(traj)
    def step(t):
        ax.cla()
        ax.plot([optimal_x[0]], [optimal_x[1]], 'o', color='green')
        #Level contours
        delta = 0.025
        x = np.arange(-1.5, 1.5, delta)
        y = np.arange(-1.5, 1.5, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = func([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [3.0, 4.0, 5.0], colors=['blue', 'purple', 'red'])

        
        # bug fixed: t -> t + 1
        ax.plot([u[0] for u in traj[:t + 1]], [u[1] for u in traj[:t + 1]], color='black')
        ax.plot([u[0] for u in traj[:t + 1]], [u[1] for u in traj[:t + 1]], 'o', color='black')
        
        fix_scaling(ax)
        ax.axis('off')
    
    plt.close() # fixed odd picture

    return FuncAnimation(fig, step,
                     frames=range(n), interval=600)

In [ ]:
# Task 2A
def func(x):
    return np.exp(x[0] + 3 * x[1]) + np.exp(x[0] - 3 * x[1]) + np.exp(-x[0])
def func_grad(x):
    return np.array([np.exp(x[0] + 3 * x[1]) + np.exp(x[0] - 3 * x[1]) - np.exp(-x[0]),
                    3 * np.exp(x[0] + 3 * x[1]) - 3 * np.exp(x[0] - 3 * x[1])])

In [ ]:
# Task 2B
def lipschitz_estimation():
    e = np.exp(1)
    e3 = np.exp(3)
    em3 = np.exp(-3)
    return np.sqrt((3 * (e * (e3 - em3) + 3 * e * (e3 + em3))) ** 2 + (3 * (e3-em3) * e + (e3 + em3) * e + e) ** 2)
                   
M = lipschitz_estimation()
print(M)

In [ ]:
# Task 2C
def make_animation(build_trajectory):
    base_animation = animate_trajectory(build_trajectory(X0))
    return HTML(base_animation.to_html5_video())  

In [ ]:
# gradient descent
def constant_step_trajectory(x0):
    alpha = 1 / M
    cur_x = x0
    trajectory = [cur_x]
    
    for i in range(iterations_number):
        cur_x = cur_x - alpha * func_grad(cur_x)
        trajectory.append(cur_x)
        
    return trajectory

make_animation(constant_step_trajectory)

In [ ]:
# Nesterov
def nesterov_trajectory(x0):
    phi = 1.0 / M
    cur_alpha = 0.9
    
    cur_x = x0
    cur_y = x0
    
    def calc_next_alpha(alpha):
        return (-alpha ** 2 + np.sqrt(alpha ** 4 + 4 * alpha ** 2)) / 2.0

    def calc_beta(cur_alpha, next_alpha):
        return (cur_alpha * (1 - cur_alpha)) / (cur_alpha ** 2 + next_alpha)
    
    trajectory = [cur_x]
    
    for i in range(iterations_number):
        next_x = cur_y - phi * func_grad(cur_y)
        next_alpha = calc_next_alpha(cur_alpha)
        beta = calc_beta(cur_alpha, next_alpha)
        cur_y, cur_x = next_x + beta * (next_x - cur_x), next_x
        cur_alpha = next_alpha
        
        trajectory.append(cur_x)
        
    return trajectory    

make_animation(nesterov_trajectory) 